In [1]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score
from numpy.lib.stride_tricks import sliding_window_view

In [2]:
train = np.load("train_split.npy", allow_pickle=True).item()
val = np.load("val_split.npy", allow_pickle=True).item()
test = np.load("test-release.npy", allow_pickle=True).item()

In [3]:
def attach_previous_frames(x, num_frames):
    if num_frames == 1:
        return x
    B, D = x.shape
    pad = np.zeros((num_frames - 1, D))
    padded = np.vstack([pad, x])
    windows = sliding_window_view(padded, window_shape=(num_frames, D)).squeeze(axis=1)
    return windows.reshape(B, -1)

In [4]:
num_frames_per_cls = 10

In [5]:
labeled_x = np.concatenate([seq["keypoints"] for seq in train["sequences"].values()], axis=0)
labeled_x = attach_previous_frames(labeled_x.reshape(-1, 28), num_frames_per_cls)
labeled_x = torch.tensor(labeled_x)
y = np.concatenate([seq["annotations"] for seq in train["sequences"].values()], axis=0)
y = torch.tensor(y)
labeled_x.shape, y.shape

(torch.Size([426635, 280]), torch.Size([426635]))

In [6]:
unlabeled_x = np.concatenate([seq["keypoints"] for seq in test["sequences"].values()], axis=0)
unlabeled_x = attach_previous_frames(unlabeled_x.reshape(-1, 28), num_frames_per_cls)
unlabeled_x = torch.tensor(unlabeled_x)
unlabeled_x.shape

torch.Size([8168491, 280])

In [7]:
std, mean = torch.std_mean(torch.cat((labeled_x, unlabeled_x), dim=0), dim=0)
labeled_x = (labeled_x - mean) / std
unlabeled_x = (unlabeled_x - mean) / std

In [8]:
val_x = np.concatenate([seq["keypoints"] for seq in val["sequences"].values()], axis=0)
val_x = attach_previous_frames(val_x.reshape(-1, 28), num_frames_per_cls)
val_x = torch.tensor(val_x)
val_x = (val_x - mean) / std
val_y = np.concatenate([seq["annotations"] for seq in val["sequences"].values()], axis=0)
val_y = torch.tensor(val_y)
val_x.shape, val_y.shape

(torch.Size([81103, 280]), torch.Size([81103]))

In [49]:
def gaussian_parameters(h, dim=-1):
    m, h = torch.split(h, h.size(dim) // 2, dim=dim)
    v = F.softplus(h) + 1e-8
    return m, v

def sample_gaussian(m, v):
    unit_sample = torch.normal(torch.zeros(m.shape), torch.ones(v.shape)).to(device)
    z = m + torch.sqrt(v) * unit_sample
    return z

def kl_cat(q, log_q, log_p):
    element_wise = (q * (log_q - log_p))
    kl = element_wise.sum(-1)
    return kl

def kl_normal(qm, qv, pm, pv):
    element_wise = 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    kl = element_wise.sum(-1)
    return kl

def log_normal(x, m, v):
    log_prob = -0.5 * (torch.log(v) + (x - m).pow(2) / v + np.log(2 * np.pi))
    log_prob = log_prob.sum(-1)
    return log_prob

def duplicate(x, rep):
    return x.expand(rep, *x.shape).reshape(-1, *x.shape[1:])

In [10]:
class Encoder(nn.Module):
    def __init__(self, z_dim, y_dim):
        super().__init__()
        self.z_dim = z_dim
        self.y_dim = y_dim
        self.net = nn.Sequential(
            nn.Linear(28 * num_frames_per_cls + y_dim, 128),
            nn.ELU(),
            nn.Linear(128, 2 * z_dim),
        )

    def encode(self, x, y):
        xy = torch.cat((x, y), dim=1)
        h = self.net(xy)
        m, v = gaussian_parameters(h, dim=1)
        return m, v

class Decoder(nn.Module):
    def __init__(self, z_dim, y_dim):
        super().__init__()
        self.z_dim = z_dim
        self.y_dim = y_dim
        self.net = nn.Sequential(
            nn.Linear(z_dim + y_dim, 128),
            nn.ELU(),
            nn.Linear(128, 28 * num_frames_per_cls)
        )

    def decode(self, z, y):
        zy = torch.cat((z, y), dim=1)
        return self.net(zy)

class Classifier(nn.Module):
    def __init__(self, y_dim):
        super().__init__()
        self.y_dim = y_dim
        self.net = nn.Sequential(
            nn.Linear(28 * num_frames_per_cls, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, y_dim)
        )

    def classify(self, x):
        return self.net(x)

In [11]:
class SSVAE(nn.Module):
    def __init__(self, gen_weight=1, class_weight=100):
        super().__init__()
        self.z_dim = 64
        self.y_dim = 4
        self.gen_weight = gen_weight
        self.class_weight = class_weight
        self.enc = Encoder(self.z_dim, self.y_dim)
        self.dec = Decoder(self.z_dim, self.y_dim)
        self.cls = Classifier(self.y_dim)

        # Set prior as fixed parameter attached to Module
        self.z_prior_m = torch.nn.Parameter(torch.zeros(1), requires_grad=False)
        self.z_prior_v = torch.nn.Parameter(torch.ones(1), requires_grad=False)

    def negative_elbo_bound(self, x):
        y_logits = self.cls.classify(x)
        y_logprob = F.log_softmax(y_logits, dim=1)
        y_prob = torch.softmax(y_logprob, dim=1) # (batch, y_dim)

        # Duplicate y based on x's batch size. Then duplicate x
        # This enumerates all possible combination of x with labels (0, 1, 2, 3)
        y = np.repeat(np.arange(self.y_dim), x.size(0))
        y = x.new(np.eye(self.y_dim)[y])
        x = duplicate(x, self.y_dim)

        z_m, z_v = self.enc.encode(x, y)
        z = sample_gaussian(z_m, z_v)
        x_m = self.dec.decode(z, y)
        x_v = torch.tensor(0.1).repeat(x_m.shape).to(device)

        kl_y = kl_cat(y_prob, y_logprob, np.log(1.0 / self.y_dim))
        kl_z = kl_normal(z_m, z_v, self.z_prior_m, self.z_prior_v)
        kl_z = (kl_z.reshape(self.y_dim, -1) * y_prob.t()).sum(dim=0)
        rec = -log_normal(x, x_m, x_v)
        rec = (rec.reshape(self.y_dim, -1) * y_prob.t()).sum(dim=0)
        nelbo = kl_y + kl_z + rec

        nelbo, kl_z, kl_y, rec = nelbo.mean(), kl_z.mean(), kl_y.mean(), rec.mean()
        return nelbo, kl_z, kl_y, rec

    def classification_cross_entropy(self, x, y):
        y_logits = self.cls.classify(x)
        return F.cross_entropy(y_logits, y.argmax(1))

    def loss(self, x, xl, yl):
        if self.gen_weight > 0:
            nelbo, kl_z, kl_y, rec = self.negative_elbo_bound(x)
        else:
            nelbo, kl_z, kl_y, rec = [0] * 4
        ce = self.classification_cross_entropy(xl, yl)
        loss = self.gen_weight * nelbo + self.class_weight * ce

        summaries = dict((
            ('train/loss', loss),
            ('class/ce', ce),
            ('gen/elbo', -nelbo),
            ('gen/kl_z', kl_z),
            ('gen/kl_y', kl_y),
            ('gen/rec', rec),
        ))

        return loss, summaries

    def compute_sigmoid_given(self, z, y):
        logits = self.dec.decode(z, y)
        return torch.sigmoid(logits)

    def sample_z(self, batch):
        return ut.sample_gaussian(self.z_prior[0].expand(batch, self.z_dim),
                                  self.z_prior[1].expand(batch, self.z_dim))

    def sample_x_given(self, z, y):
        return torch.bernoulli(self.compute_sigmoid_given(z, y))


In [12]:
epochs = 20
device = "cuda"
labeled_batch_size = 64
unlabeled_batch_size = 320
learning_rate = 1e-3
seed = 1

In [13]:
torch.manual_seed(seed);
labeled_dataset = TensorDataset(labeled_x, y)
unlabeled_dataset = TensorDataset(unlabeled_x)
labeled_loader = DataLoader(labeled_dataset, batch_size=labeled_batch_size, shuffle=True)

In [14]:
model = SSVAE(gen_weight=0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(epochs):
    total_loss = 0
    total_count = 0
    for i, (xl, yl) in enumerate(labeled_loader):
        optimizer.zero_grad()

        unlabeled_indices = torch.randint(0, len(unlabeled_dataset), (unlabeled_batch_size,))
        xu = unlabeled_dataset[unlabeled_indices][0]

        xu = xu.to(device).float()
        yl = yl.new(np.eye(4)[yl]).to(device).float()
        xl = xl.to(device).float()
        loss, summaries = model.loss(xu, xl, yl)
    
        loss.backward()
        optimizer.step()

        total_loss += loss.detach().item() * len(xl)
        total_count += len(xl)

        if i % 3000 == 0:
            print(f"Train_loss={total_loss / total_count:.4f}")

    train_pred = model.cls.classify(labeled_x.to(device).float()).argmax(1)
    train_f1 = f1_score(y, train_pred.cpu(), average="macro", labels=[0, 1, 2])
    val_pred = model.cls.classify(val_x.to(device).float()).argmax(1)
    val_f1 = f1_score(val_y, val_pred.cpu(), average="macro", labels=[0, 1, 2])

    print(f"Epoch {epoch}, Train-F1={train_f1:.4f}, Val-F1={val_f1:.4f}")

Train_loss=141.3575
Train_loss=38.6211
Train_loss=35.8927
Epoch 0, Train-F1=0.6874, Val-F1=0.5539
Train_loss=17.6223
Train_loss=30.6116
Train_loss=29.9602
Epoch 1, Train-F1=0.7447, Val-F1=0.5528
Train_loss=27.4175
Train_loss=27.9577
Train_loss=27.5408
Epoch 2, Train-F1=0.7720, Val-F1=0.5689
Train_loss=16.4001
Train_loss=26.1178
Train_loss=25.9384
Epoch 3, Train-F1=0.7822, Val-F1=0.5175
Train_loss=24.8212
Train_loss=24.8936
Train_loss=24.7009
Epoch 4, Train-F1=0.7714, Val-F1=0.5335
Train_loss=19.8337
Train_loss=23.7141
Train_loss=23.7532
Epoch 5, Train-F1=0.8136, Val-F1=0.5702
Train_loss=29.4009
Train_loss=22.7871
Train_loss=22.8677
Epoch 6, Train-F1=0.7995, Val-F1=0.4858
Train_loss=20.3117
Train_loss=22.3570
Train_loss=22.1598
Epoch 7, Train-F1=0.8256, Val-F1=0.5431
Train_loss=18.9684
Train_loss=21.5382
Train_loss=21.5264
Epoch 8, Train-F1=0.8193, Val-F1=0.5411
Train_loss=28.0795
Train_loss=21.0183
Train_loss=20.9491
Epoch 9, Train-F1=0.8365, Val-F1=0.5007
Train_loss=20.4425
Train_loss

In [16]:
model = SSVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(epochs):
    total_loss = 0
    total_count = 0
    for i, (xl, yl) in enumerate(labeled_loader):
        optimizer.zero_grad()

        unlabeled_indices = torch.randint(0, len(unlabeled_dataset), (unlabeled_batch_size,))
        xu = unlabeled_dataset[unlabeled_indices][0]

        xu = xu.to(device).float()
        yl = yl.new(np.eye(4)[yl]).to(device).float()
        xl = xl.to(device).float()
        loss, summaries = model.loss(xu, xl, yl)
    
        loss.backward()
        optimizer.step()

        total_loss += loss.detach().item() * len(xl)
        total_count += len(xl)

        if i % 3000 == 0:
            print(f"Train_loss={total_loss / total_count:.4f}")

    train_pred = model.cls.classify(labeled_x.to(device).float()).argmax(1)
    train_f1 = f1_score(y, train_pred.cpu(), average="macro", labels=[0, 1, 2])
    val_pred = model.cls.classify(val_x.to(device).float()).argmax(1)
    val_f1 = f1_score(val_y, val_pred.cpu(), average="macro", labels=[0, 1, 2])

    print(f"Epoch {epoch}, Train-F1={train_f1:.4f}, Val-F1={val_f1:.4f}")

Train_loss=1575.3906
Train_loss=32.9907
Train_loss=20.4748
Epoch 0, Train-F1=0.6804, Val-F1=0.5475
Train_loss=4.6729
Train_loss=4.4974
Train_loss=3.4594
Epoch 1, Train-F1=0.7180, Val-F1=0.5439
Train_loss=4.6663
Train_loss=0.7796
Train_loss=0.2835
Epoch 2, Train-F1=0.7568, Val-F1=0.5276
Train_loss=11.3763
Train_loss=-1.7113
Train_loss=-2.0872
Epoch 3, Train-F1=0.7788, Val-F1=0.4970
Train_loss=-2.3672
Train_loss=-3.5126
Train_loss=-3.7839
Epoch 4, Train-F1=0.7886, Val-F1=0.5842
Train_loss=-9.9792
Train_loss=-4.9361
Train_loss=-5.2442
Epoch 5, Train-F1=0.8024, Val-F1=0.5250
Train_loss=-15.8496
Train_loss=-6.1993
Train_loss=-6.2841
Epoch 6, Train-F1=0.8099, Val-F1=0.5327
Train_loss=7.3044
Train_loss=-7.0621
Train_loss=-7.1523
Epoch 7, Train-F1=0.8138, Val-F1=0.5611
Train_loss=-7.7760
Train_loss=-7.7542
Train_loss=-7.9260
Epoch 8, Train-F1=0.8185, Val-F1=0.5486
Train_loss=3.4565
Train_loss=-8.5334
Train_loss=-8.6773
Epoch 9, Train-F1=0.8278, Val-F1=0.5319
Train_loss=-8.7251
Train_loss=-9.23

KeyboardInterrupt: 